# Analysis code for the paper "B1+ shimming in the cervical spinal cord at 7T"

## Data

The data can be download at: XXX (TODO: upload on openneuro)

The structure of the input dataset is as follows:
~~~
RF_shimming_project_cleanupload/
├── SubA
│   ├── GRE_B1
│   │   ├── CVred
│   │   ├── Noshim
│   │   ├── PatSpec
│   │   ├── PhaseOnly
│   │   ├── SAReff
│   │   ├── Target
│   │   └── VolSpec
│   └── MPRAGE
├── SubB
├── SubC
├── SubD
└── SubE
~~~


## Overview of processing pipeline

For each subject:

- Segment the SC from the GRE scan
- Extract the signal intensity within this mask from the GRE scans
- Coregister the TFL_B1map to the corresponding GRE scan from each subfolder (CVred, Noshim, etc.)
- Apply the computed warping field to bring the SC segmentation to the TFL_B1map
- Convert the B1+ maps to nT/V units
- Extract the B1+ value within the registered mask

In [ ]:
# Necessary imports

import os
import re
import json
import subprocess
import glob
import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image
# from IPython.display import display
# from tabulate import tabulate
import nibabel as nib
# import pandas as pd

In [ ]:
# Download data

!datalad install https://github.com/OpenNeuroDatasets/ds004906.git
os.chdir("ds004906")
# !datalad get . # uncomment for production
!datalad get sub-01/  # comment for debugging
path_data = os.getcwd()

In [ ]:
# Run segmentation on GRE scan

subjects = glob.glob("sub-*")
for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_deepseg_sc -i {subject}_acq-CoV_T2starw.nii.gz -c t2 -qc {os.path.join(path_data, "qc")}

In [ ]:
# Register TFL B1maps to the GRE scan

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    !sct_register_multimodal -i {subject}_acq-anatCoV_TB1TFL.nii.gz -d ../anat/{subject}_acq-CoV_T2starw.nii.gz -param step=1,type=im,algo=slicereg,metric=CC -qc {os.path.join(path_data, "qc")}